# Vectorization and Modeling Phases
### Group 03:
- Catarina Oliveira | 20211616
- Daniel Kruk | 20211687
- Joana Rosa | 20211516
- Marcelo Junior | 20211677
- Martim Serra | 20211543

##### This notebook includes:
 - Vectorization of the data and vocabulary chosen;

 - Defining and training a model;

 - Creating predictions and evaluating them;

## Libraries

In [1]:
%load_ext autoreload
%autoreload 2
from functions import *
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import gc
gc.collect()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


DONE


0

### Vocabulary

In [2]:
from vocabulary import vocabulary 
from vocabulary1 import vocabulary1
from vocabulary4 import vocabulary4 
from vocabulary9 import vocabulary9
from vocabulary10 import vocabulary10 
from vocabulary11 import vocabulary11


# join vocabularies
big_vocabulary = set(vocabulary + vocabulary1 +
                     vocabulary4 + vocabulary9 +
                     vocabulary10 + vocabulary11)
len(big_vocabulary)

4495

### Importing the Data

In [3]:
data = pd.read_csv('data/preproc_final/train_preproc.csv', index_col= 'id')
test_data = pd.read_csv('data/preproc_final/test_preproc.csv', index_col= 'id').sort_index()
sample_submission = pd.read_csv('data/sample_submission.csv', index_col= 'id').sort_index()

### Vectorizer

In [6]:
# TDIF
tfidf = TfidfVectorizer(sublinear_tf=True,
                        min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english', 
                        vocabulary=set(big_vocabulary)
                        )
#train the vectorizer
train_features = tfidf.fit_transform(data.lyrics).toarray()
train_labels = data.tag

X_test = tfidf.transform(test_data.lyrics).toarray()

### Splitting the data

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_features, 
                                                               train_labels, 
                                                               test_size=0.25, 
                                                               random_state=42)

# Model
 Defining and fitting the training data to the model

In [8]:
model = LogisticRegression(random_state=42, C = 100)

model.fit(X_train, y_train)

c:\Users\Martin\anaconda3\envs\project\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100, random_state=42)

Creating predictions

In [9]:
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

Evaluating the predications

In [10]:
score_report(y_train, train_pred, y_val, val_pred)

──────────────────────────────────────────────────────────────────
                            TRAIN                                                 
-----------------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

     country       0.56      0.22      0.32      2591
        misc       0.81      0.53      0.64      4185
         pop       0.64      0.83      0.72     41588
         rap       0.87      0.89      0.88     29146
          rb       0.54      0.20      0.29      4634
        rock       0.58      0.39      0.46     18906

    accuracy                           0.70    101050
   macro avg       0.67      0.51      0.55    101050
weighted avg       0.70      0.70      0.69    101050

─────────────────────────────────────────────────────────────────
                            VALIDATION                                                 
---------------------------------------------

Getting the test set predictions and storing them in the final file

In [ ]:
final_pred = model.predict(X_test)

sample_submission['tag'] = final_pred
sample_submission.to_csv('data/Group03_Version46.csv')